In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.metrics import Accuracy, F1Score
import os
import cv2
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_folders = [
    '/content/drive/MyDrive/flowers_recognition/flowers/sunflower',
    '/content/drive/MyDrive/flowers_recognition/flowers/tulip',
    '/content/drive/MyDrive/flowers_recognition/flowers/dandelion',
    '/content/drive/MyDrive/flowers_recognition/flowers/rose',
    '/content/drive/MyDrive/flowers_recognition/flowers/daisy'
]

for i, folder in enumerate(data_folders, start=1):
    num_files = len(os.listdir(folder))
    print(f"Dataset {i}: Počet obrázků = {num_files}")

Dataset 1: Počet obrázků = 733
Dataset 2: Počet obrázků = 984
Dataset 3: Počet obrázků = 1052
Dataset 4: Počet obrázků = 784
Dataset 5: Počet obrázků = 764


In [ ]:
# Cesta k datům
data_dir = '/content/drive/MyDrive/flowers_recognition/flowers'

# Cílová velikost obrázků
target_size = (224, 224)

# Vytvoření generátorů pro trénovací a validační data
datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    class_mode='categorical',
    subset='validation'
)

Found 3457 images belonging to 5 classes.
Found 860 images belonging to 5 classes.


In [ ]:
target_size = (224, 224)

def prepare_image(image_path, target_size):
    image = cv2.imread(image_path)
    original_height, original_width, _ = image.shape

    # Určení poměru stran pro zmenšení na cílový rozměr
    width_ratio = target_size[0] / original_width
    height_ratio = target_size[1] / original_height

    # Určení menšího poměru pro zachování původního poměru stran
    min_ratio = min(width_ratio, height_ratio)

    # Výpočet nových rozměrů
    new_width = int(original_width * min_ratio)
    new_height = int(original_height * min_ratio)

    # Zmenšení obrázku
    resized_image = cv2.resize(image, (new_width, new_height))

    # Vytvoření prázdného obrázku cílové velikosti
    final_image = np.zeros((*target_size, 3), dtype=np.uint8)

    # Umístění zmenšeného obrázku do středu cílového obrázku
    start_x = (target_size[0] - new_width) // 2
    start_y = (target_size[1] - new_height) // 2
    final_image[start_y:start_y+new_height, start_x:start_x+new_width] = resized_image

    return final_image

# Načítání a příprava obrázků
image_list = []
label_list = []

class_names = os.listdir(data_dir)

for class_name in class_names:
    class_dir = os.path.join(data_dir, class_name)
    for image_name in os.listdir(class_dir):
        if image_name.endswith('.jpg'):
            image_path = os.path.join(class_dir, image_name)
            prepared_image = prepare_image(image_path, target_size)
            image_list.append(prepared_image)
            label_list.append(class_name)

# Převedení na numpy pole
images = np.array(image_list)
labels = np.array(label_list)


In [ ]:
# Vytvoření modelu
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))

    #FC layers

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

# Kompilace modelu
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Trénování modelu
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(train_gen, epochs=15, validation_data=val_gen, callbacks=[early_stopping])

# Evaluate the model on the validation generator
val_loss, val_acc = model.evaluate(val_gen)
print(f"Validation accuracy: {val_acc}")


# Summary of the model
model.summary()

Epoch 1/15
109/109 [==============================] - 2124s 19s/step - loss: 1.6032 - accuracy: 0.2308 - val_loss: 1.6004 - val_accuracy: 0.2442
Epoch 2/15
109/109 [==============================] - 2079s 19s/step - loss: 1.6013 - accuracy: 0.2395 - val_loss: 1.5992 - val_accuracy: 0.2279
Epoch 3/15
109/109 [==============================] - ETA: 0s - loss: 1.5996 - accuracy: 0.2517 